# Análisis Exploratorio de Datos - Resumen Estadistico Mensual (REM)
Este notebook tiene como objetivo explorar una base de datos que contiene el resumen estadistico mensual de todas las atenciones en Chile correspondientes al año 2024 en formato `.csv`, preparar los datos y generar un dataset para análisis posteriores.

## Importar librerias
Inicialmente importamos las librerias necesarias para el funcionamiento del notebook.

In [1]:
import pandas as pd

## Lectura archivo: REM
Se lee el archivo de REM utilizando pandas.<br>
El archivo REM se obtiene desde el siguiente [enlace](https://deis.minsal.cl/#datosabiertos), datos abiertos, serie REM. Se debe descargar el ZIP completo el cual contiene todos los distintos tipos de REM disponibles y un diccionaro indicando los distintos codigos que lo componen. Para nuestro caso de estudio se ocupara la Serie A04 que contiene informacion relevante para el estudio (Consultas y otras atenciones en la red).<br>
Se puede consultar el manual de Series REM en el siguiente [enlace](https://repositoriodeis.minsal.cl/ContenidoSitioWeb2020/REM/2025/SERIE/Manual%20Series%20REM%202025%20-2026%20SERIE%20A%20-BS-BM-%20DV1.2.pdf), el cual contiene informacion detallada de distintas indicaciones para comprender el registro estadistico y los codigos que lo componen.

In [2]:
# Ruta al archivo 
ruta = '../../data/raw_data/REMserieA2024.csv'

# Leer el CSV
df = pd.read_csv(ruta,sep=';',encoding='latin1')

# Ver las primeras filas
df.head()

,Mes,IdServicio,Ano,IdEstablecimiento,CodigoPrestacion,IdRegion,IdComuna,Col01,Col02,Col03,...,Col41,Col42,Col43,Col44,Col45,Col46,Col47,Col48,Col49,Col50
0,10,5,2024,105100,09600394,4,4101,182.0,94.0,64.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10,5,2024,105100,09600412,4,4101,3.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
2,10,5,2024,105100,11052700,4,4101,2.0,0.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
3,10,5,2024,105100,11052800,4,4101,2.0,1.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
4,10,5,2024,105100,11053000,4,4101,137.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,NaN,NaN


## Limpieza Valores Nulos
Se debe rellenar los valores que esten vacios por el valor 0.

In [3]:
df=df.fillna(0)

## Especificacion codigos de interes
A continuacion se especificaran codigos que seran de interes para el estudio.

### REM A04 - Sección A: Consultas Médicas
Esta sección registra las consultas realizadas por médicos, desglosadas por diagnóstico o especialidad.

#### Filas (Prestaciones)
| Código | Descripción |
| :--- | :--- |
| 03020101 | IRA Alta |
| 03020201 | Síndrome Bronquial Obstructivo |
| 03020301 | Neumonía |
| 03020402 | Exacerbación Asma |
| 03020403 | Enfermedad Pulmonar Obstructiva Crónica |
| 03020401 | Otras respiratorias |
| 03040210 | Obstétrica |
| 03040220 | Ginecológica |
| 04040100 | Ginecológica por Infertilidad |
| 04025010 | Infección Transmisión Sexual |
| 04025020 | VIH-SIDA |
| 04025025 | Salud Mental |
| 04040427 | Cardiovascular |
| 03020501 | Otras Morbilidades |

#### Columnas
* **COL01:** Total, Ambos sexos
* **COL41:** Total, Migrantes

---

### REM A04 - Sección B: Consultas de Profesionales No Médicos
Esta sección registra las atenciones realizadas por otros profesionales de la salud.

#### Filas (Prestaciones)

| Código | Descripción |
| :--- | :--- |
| 03020702 | Enfermera/o |
| 04025030 | Matrona/ón (Morb. Ginecológica) |
| 04025040 | Matrona/ón (ITS) |
| 04040200 | Matrona/ón (Infertilidad) |
| 04025050 | Matrona/ón (Otras consultas) |
| 04050100 | Matrona/ón (Salud sexual) |
| 09600278 | Matrona/ón (Piso pélvico) |
| 03020604 | Nutricionista (Otras consultas) |
| 04050110 | Nutricionista malnutrición por exceso |
| 04050120 | Nutricionista malnutrición por déficit |
| 03020908 | Psicólogo/a |
| 03030250 | Fonoaudiólogo/a |
| 03030270 | Terapeuta Ocupacional |
| 03030280 | Tecnólogo/a Médico/a (Excluye UAPO) |
| 03020806 | Trabajador/a Social (Excluye Salud Mental) |
| 04070100 | Educadora de Párvulos |

#### Columnas
* **COL01:** Total, Ambos sexos
* **COL41:** Total, Migrantes

In [4]:
"""
-------
REM-A07
-------
SECCIÓN A : CONSULTAS MÉDICAS DE ESPECIALIDAD
07020130	Pediatría
07020230	Medicina Interna
07020330	Neonatología
07020331	Enfermedad Respiratoria Pediátrica (Broncopulmonar Infantil)
07020332	Enfermedad Respiratoria de Adulto (Broncopulmonar)
07024219	Cardiología Pediátrica
07020500	Cardiología Adulto
07020501	Endocrinología Pediátrica
07020600	Endocrinología Adulto
07020601	Gastroenterología Pediátrica
07020700	Gastroenterología Adulto
07020800	Genética Clínica
07020801	Hemato-Oncología Infantil
07020900	Hematología Adulto
07020901	Nefrología Pediátrica
07021000	Nefrología Adulto
07021001	Nutriólogo Pediátrico
07021100	Nutriólogo Adulto
07021101	Reumatología Pediátrica
07021230	Reumatología Adulto
07021300	Dermatología
07021301	Infectología Pediátrica
07022000	Infectología Adulto
07022001	Inmunología
07021531	Geriatría
07022132	Medicina Física y Rehabilitación Pediátrica (Fisiatría Pediátrica)
07022133	Medicina Física y Rehabilitación Adulto (Fisiatría Adulto)
07022134	Neurología Pediátrica
07021700	Neurología Adulto
07021800	Oncología Médica
07021801	Psiquiatría Pediátrica y de La Adolescencia
07021900	Psiquiatría Adulto
07022130	Cirugía Pediátrica
07022142	Cirugía General Adulto
07022143	Cirugía Digestiva (Alta)
07022144	Cirugía de Cabeza, Cuello y Maxilofacial
07022135	Cirugía Plástica y Reparadora Pediátrica
07022136	Cirugía Plástica y Reparadora Adulto
07022137	Coloproctología (Cirugía Digestiva Baja)
07022700	Cirugía Tórax
07022800	Cirugía Vascular Periférica
07022900	Neurocirugía
07021701	Cirugía Cardiovascular
07023100	Anestesiología
07023200	Obstetricia
07023201	Ginecología Pediátrica y de la Adolescencia
07023202	Ginecología Adulto
07023203	Oftalmología
07023700	Otorrinolaringología
07023701	Traumatología y Ortopedia Pediátrica
07023702	Traumatología y Ortopedia Adulto
07023703	Urología Pediátrica
07024000	Urología Adulto
07024001	Medicina Familiar del Niño
07024200	Medicina Familiar
07030500	Medicina Familiar Adulto
07024201	Diabetología
07024202	Medicina Nuclear (Excluye Informes)
07030501	Imagenología
07030502	Radioterapia Oncológica
--------
Columnas
COL01       Total
"""

'\n-------\nREM-A07\n-------\nSECCIÓN A : CONSULTAS MÉDICAS DE ESPECIALIDAD\n07020130\tPediatría\n07020230\tMedicina Interna\n07020330\tNeonatología\n07020331\tEnfermedad Respiratoria Pediátrica (Broncopulmonar Infantil)\n07020332\tEnfermedad Respiratoria de Adulto (Broncopulmonar)\n07024219\tCardiología Pediátrica\n07020500\tCardiología Adulto\n07020501\tEndocrinología Pediátrica\n07020600\tEndocrinología Adulto\n07020601\tGastroenterología Pediátrica\n07020700\tGastroenterología Adulto\n07020800\tGenética Clínica\n07020801\tHemato-Oncología Infantil\n07020900\tHematología Adulto\n07020901\tNefrología Pediátrica\n07021000\tNefrología Adulto\n07021001\tNutriólogo Pediátrico\n07021100\tNutriólogo Adulto\n07021101\tReumatología Pediátrica\n07021230\tReumatología Adulto\n07021300\tDermatología\n07021301\tInfectología Pediátrica\n07022000\tInfectología Adulto\n07022001\tInmunología\n07021531\tGeriatría\n07022132\tMedicina Física y Rehabilitación Pediátrica (Fisiatría Pediátrica)\n07022133\

## Asignacion a variables
Se asignan los codigos de interes a variables, para nuestro caso se seleccionara solamente atenciones correspondiente a Serie REM A04 A y B.

In [5]:
REM_A04_A = ["03020101",
             "03020201",
             "03020301",
             "03020402",
             "03020403",
             "03020401",
             "03040210",
             "03040220",
             "04040100",
             "04025010",
             "04025020",
             "04025025",
             "04040427",
             "03020501"]

REM_A04_B = ["03020702",
             "04025030",
             "04025040",
             "04040200",
             "04025050",
             "04050100",
             "09600278",
             "03020604",
             "04050110",
             "04050120",
             "03020908",
             "03030250",
             "03030270",
             "03030280",
             "03020806",
             "04070100"]

## Se filtra segun los codigos
Se crea un dataframe que contiene solo los codigos especificados anteriormente.

In [6]:
# Combinar ambas listas
codigos_validos = REM_A04_A + REM_A04_B

# Filtrar dataframe
df_filtrado = df[df["CodigoPrestacion"].isin(codigos_validos)]

# Seleccionar solo las columnas necesarias
df_filtrado = df_filtrado[
    ["Mes", "Ano", "IdEstablecimiento", 
     "CodigoPrestacion", "IdRegion", "IdComuna", "Col01", "Col41"]
]

# Renombramos columnas
df_filtrado = df_filtrado.rename(columns={
    "Col01": "total_pacientes",
    "Col41": "total_migrantes"
})

df_filtrado.head()

,Mes,Ano,IdEstablecimiento,CodigoPrestacion,IdRegion,IdComuna,total_pacientes,total_migrantes
92,4,2024,105700,03020604,4,4101,2.0,0.0
138,3,2024,105702,04025030,4,4101,3.0,0.0
141,5,2024,105700,03020501,4,4101,94.0,3.0
142,5,2024,105700,03020702,4,4101,8.0,0.0
150,5,2024,105700,04025030,4,4101,13.0,0.0


## Agrupacion
Agruparemos los codigos correspondientes a Serie REM A04 A y B, a los cuales se le asignaran nombres de Consulta Medica y Consulta No Medica, obteniendo el detalle del total de atenciones por comuna mensual y del total de atenciones a migrantes.

In [7]:

# Etiquetar segun lista
df_filtrado["GrupoPrestacion"] = df_filtrado["CodigoPrestacion"].apply(
    lambda x: "Consulta Medica" if x in REM_A04_A else ("Consulta no Medica" if x in REM_A04_B else "Otro")
)

# Agrupar
df_grouped = (
    df_filtrado
    .groupby(["Mes", "Ano","IdEstablecimiento", "GrupoPrestacion", "IdRegion", "IdComuna"], as_index=False)
    .agg({
        "total_pacientes": "sum",
        "total_migrantes": "sum"
    })
)
df_grouped.head()

,Mes,Ano,IdEstablecimiento,GrupoPrestacion,IdRegion,IdComuna,total_pacientes,total_migrantes
0,1,2024,101011,Consulta Medica,15,15101,62.0,0.0
1,1,2024,101030,Consulta Medica,15,15101,586.0,41.0
2,1,2024,101030,Consulta no Medica,15,15101,84.0,1.0
3,1,2024,101100,Consulta Medica,15,15101,586.0,41.0
4,1,2024,101100,Consulta no Medica,15,15101,84.0,1.0


## Exportacion a CSV total de atenciones
Finalmente exportamos a un csv la informacion correspondiente al total de atenciones por comuna.

In [8]:
df_grouped.to_csv('../../data/processed_data/total_atenciones.csv', index=False)